# 对装饰器进行解包装
- 我们已经把装饰器添加到一个函数上了，但是想撤销它，访问未经包装过的那个原始函数

## 假设装饰器中已经使用了@wraps，一般来说，我们可以通过访问```__wrapped__```属性来获取对原始函数的访问

In [1]:
import time
from functools import wraps


def timethis(func):
    '''
     Decorator that reports the execution time.
    '''

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return result
    return wrapper

In [2]:
@timethis
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

In [3]:
countdown.__wrapped__(50000)

In [4]:
countdown(50000)

countdown 0.0030078887939453125


## 直接访问装饰器背后的未被包装的函数对于调试、反射（introspection）以及其他一些涉及函数的操作很有帮助。但是，本节讨论的技术只有在实现装饰器时利用functools模块中的@wraps对元数据进行了适当的拷贝，或者直接设定了```__wrapped__```属性时才有用。

** 如果有多个装饰器已经作用在某个函数上了，那么访问```__wrapped__```属性的行为目前是未定义的，应该避免这种情况。**

In [5]:
def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Decorator 1")
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Decorator 2")
        return func(*args, **kwargs)
    return wrapper

def decorator3(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Decorator 3")
        return func(*args, **kwargs)
    return wrapper

In [6]:
@decorator1
@decorator2
@decorator3
def add(x, y):
    return x + y

In [7]:
add(2, 3)

Decorator 1
Decorator 2
Decorator 3


5

In [8]:
add.__wrapped__(2, 3)

Decorator 2
Decorator 3


5

In [9]:
add.__wrapped__.__wrapped__(2, 3)

Decorator 3


5

In [10]:
add.__wrapped__.__wrapped__.__wrapped__(2, 3)

5

In [11]:
for i in range(2):
    add = add.__wrapped__

In [12]:
add(2, 3)

Decorator 3


5

** 可见在python3.6版本中，一个方法有多个装饰器时，调用```__wrapped__```属性每次只解除最外层的包装，即定义时最上面的包装**

## 最后需要注意的是，并不是所有的装饰器都使用了@wraps，因此有些装饰器的行为可能与我们所期望的有所区别，特别是内建的装饰器@staticmethod和@classmethod创建的描述符对象（descriptor）并不遵循这个约定（相反，会把原始函数保存在```__func__```属性中），所以，具体问题需要具体分析，每个人遇到的情况可能不同。